<a href="https://colab.research.google.com/github/ellegonzalez/dataparksearch/blob/master/VoxKelseyPiperResize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple Demo of 'Progressive Face Super-Resolution via Attention to Facial Landmark' 
Original Paper: https://arxiv.org/abs/1908.08239

I just did the bare minimum needed to try it.

[https://secure.wphackedhelp.com/blog/](https://secure.wphackedhelp.com/blog/)

Change RunTime type to GPU and Python 3

Put images in /content/Progressive-Face-Super-Resolution/dataset/images_to_resize after the directory is created.

Results will be in:

/content/Progressive-Face-Super-Resolution/result

In [ ]:
!git clone https://github.com/DeokyunKim/Progressive-Face-Super-Resolution.git
%cd Progressive-Face-Super-Resolution/
!mkdir /content/Progressive-Face-Super-Resolution/dataset
!mkdir /content/Progressive-Face-Super-Resolution/dataset/images_to_resize

In [ ]:
!wget https://i.imgur.com/HpkjoOM.png -P /content/Progressive-Face-Super-Resolution/dataset/images_to_resize #fullsize
!wget https://i.imgur.com/2w2Y6li.png -P /content/Progressive-Face-Super-Resolution/dataset/images_to_resize #16x16 resizede externally

In [ ]:
#Setup Training Celeba
#This will not work without tweaking other things, but I left it here in case people find it useful

#%cd /content/Progressive-Face-Super-Resolution
#!mkdir /content/Progressive-Face-Super-Resolution/dataset
#!mkdir /content/Progressive-Face-Super-Resolution/dataset/CelebA
#!mkdir /content/Progressive-Face-Super-Resolution/dataset/CelebA/Img
#!mkdir /content/Progressive-Face-Super-Resolution/dataset/CelebA/Anno


#!gdown https://drive.google.com/uc?id=0B7EVK8r0v71pZjFTYXZWM3FlRnM
#!unzip /content/Progressive-Face-Super-Resolution/img_align_celeba.zip
#!mv /content/Progressive-Face-Super-Resolution/dataset/img_align_celeba /content/Progressive-Face-Super-Resolution/dataset/CelebA/Img
#!mv /content/Progressive-Face-Super-Resolution/list_eval_partition.txt /content/Progressive-Face-Super-Resolution/dataset/CelebA/Anno
#!mv /content/Progressive-Face-Super-Resolution/dataset/CelebA/Anno /content/Progressive-Face-Super-Resolution/dataset/
#!gdown https://drive.google.com/uc?id=0B7EVK8r0v71pY0NSMzRuSXJEVkk

In [ ]:
%%writefile dataloader.py
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms
from os.path import join
from PIL import Image
import glob


#I just ripped out the CelebA stuff, it does work though (check the github for the origial version of this file for that)
class CelebDataSet(Dataset):
    """CelebA dataset
    Parameters:
        data_path (str)     -- CelebA dataset main directory(inculduing '/Img' and '/Anno') path
        state (str)         -- dataset phase 'train' | 'val' | 'test'

    Center crop the alingned celeb dataset to 178x178 to include the face area and then downsample to 128x128(Step3).
    In addition, for progressive training, the target image for each step is resized to 32x32(Step1) and 64x64(Step2).
    """

    def __init__(self, data_path = './dataset/', state = 'train'):
        self.main_path = data_path
        self.state = state

        #self.img_path = join(self.main_path, 'CelebA/Img/img_align_celeba')
        self.img_path = join(self.main_path, '/content/Progressive-Face-Super-Resolution/dataset/images_to_resize')
        #self.eval_partition_path = join(self.main_path, 'Anno/list_eval_partition.txt')

        train_img_list = []
        val_img_list = []
        test_img_list = []

        image_list = []
        for filename in sorted(glob.glob('/content/Progressive-Face-Super-Resolution/dataset/images_to_resize/*.png')): 
            #im=Image.open(filename)
            image_list.append(filename)
            
        for filename in sorted(glob.glob('/content/Progressive-Face-Super-Resolution/dataset/images_to_resize/*.jpg')): 
            #im=Image.open(filename)
            image_list.append(filename)
            
        self.image_list = image_list


        #image center cropping
        #Let's just use whatever we get 
        self.pre_process = transforms.Compose([
                                    #These steps are used when training on Celeba dataset
                                    #transforms.Resize((178, 178)),
                                    #transforms.CenterCrop((178, 178)),
                                    transforms.Resize((128, 128)),
                                    ])

        self.totensor = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                    ])

        self._64x64_down_sampling = transforms.Resize((64, 64))
        self._32x32_down_sampling = transforms.Resize((32, 32))
        self._16x16_down_sampling = transforms.Resize((16,16))

    #You can modify this processing chain if your inputs are 16x16, though I actualy find pixel art tends to upscale better if with it in place
    #I usually tried various options and picked the best
    #(resized UP and then back down blurns the pixels, allows for more model creativity it seems...)
    #I also iteratively ran inputs back in, which was really effective with emoji and pixel sprites who usually grew eyes in 2 or 3 iterations, but that code is a mess and not in this notebook
    def __getitem__(self, index):
                
        image_path = join(self.img_path, self.image_list[index])
        target_image = Image.open(image_path).convert('RGB')
        target_image = self.pre_process(target_image)
        x4_target_image = self._64x64_down_sampling(target_image)
        x2_target_image = self._32x32_down_sampling(x4_target_image)
        input_image = self._16x16_down_sampling(x2_target_image)

        x2_target_image = self.totensor(x2_target_image)
        x4_target_image = self.totensor(x4_target_image)
        target_image = self.totensor(target_image)
        input_image = self.totensor(input_image)
       

        return x2_target_image, x4_target_image, target_image, input_image
   

    def __len__(self):
        return len(self.image_list)

Overwriting dataloader.py


In [ ]:
%%writefile eval.py

#The only reason this file is here to fix a typo, though it's helpful for quick changes too, this version also outputs the input and ground truth images (which may be resized UP if in put is pixel art)
import torch
from torch import optim, nn
import argparse
from dataloader import CelebDataSet
from torch.utils.data import DataLoader
from model import Generator
import os
from torch.autograd import Variable, grad
import sys
from torchvision import utils
from math import log10
from ssim import ssim, msssim

def test(dataloader, generator, MSE_Loss, step, alpha):
    avg_psnr = 0
    avg_ssim = 0
    avg_msssim = 0
    for i, (x2_target_image, x4_target_image, target_image, input_image) in enumerate(dataloader):
        if step==1:
            target_image = x2_target_image.to(device)
        elif step==2:
            target_image = x4_target_image.to(device)
        else:
            target_image = target_image.to(device)

        input_image = input_image.to(device)
        predicted_image = generator(input_image, step, alpha)
        mse_loss = MSE_Loss(0.5*predicted_image+0.5, 0.5*target_image+0.5)
        psnr = 10*log10(1./mse_loss.item())
        avg_psnr += psnr
        _ssim = ssim(0.5*predicted_image+0.5, 0.5*target_image+0.5)
        avg_ssim += _ssim.item()
        ms_ssim = msssim(0.5*predicted_image+0.5, 0.5*target_image+0.5)
        avg_msssim += ms_ssim.item()

        sys.stdout.write('\r [%d/%d] Test progress... PSNR: %6.4f'%(i, len(dataloader), psnr))
        #Note the missing parens in the origina
        utils.save_image(0.5*predicted_image+0.5, os.path.join(args.result_path, '%d_predictedresults.png'%i))      
        utils.save_image(0.5*target_image+0.5, os.path.join(args.result_path, '%d_ground_truth'%i  + '.png' ))
        utils.save_image(0.5*input_image+0.5, os.path.join(args.result_path, '%d_16x16input'%i + '.png' ))   
        
    print('Test done, Average PSNR:%6.4f, Average SSIM:%6.4f, Average MS-SSIM:%6.4f '%(avg_psnr/len(dataloader),avg_ssim/len(dataloader), avg_msssim/len(dataloader)))


if __name__ == '__main__':
    parser = argparse.ArgumentParser('Implemnetation of Progressive Face Super-Resolution Attention to Face Landmarks')
    parser.add_argument('--batch-size', default=16, type=int)
    parser.add_argument('--checkpoint-path', default='./checkpoints/', type=str)
    parser.add_argument('--data-path', default='./dataset/', type=str)
    parser.add_argument('--result-path', default='./result/', type=str)
    parser.add_argument('--workers', default=4, type=int)
    args = parser.parse_args()

    if not os.path.exists(args.result_path):
        os.mkdir(args.result_path)
        print('===>make directory', args.result_path)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = CelebDataSet(data_path=args.data_path, state='test')
    dataloader = DataLoader(dataset=dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.workers, pin_memory=True)

    generator = Generator().to(device)
    g_checkpoint = torch.load(args.checkpoint_path)
    generator.load_state_dict(g_checkpoint['model_state_dict'], strict=False)
    step = g_checkpoint['step']
    alpha = g_checkpoint['alpha']
    iteration = g_checkpoint['iteration']

    print('pre-trained model is loaded step:%d, iteration:%d'%(step, iteration))
    MSE_Loss = nn.MSELoss()

    test(dataloader, generator, MSE_Loss, step, alpha)

Overwriting eval.py


In [ ]:
%cd /content/Progressive-Face-Super-Resolution
!python3 eval.py --data-path './dataset' --checkpoint-path '/content/Progressive-Face-Super-Resolution/checkpoints/generator_checkpoint.ckpt'

## Celeb Dataset is 178 X 212, their Dataloader takes a center crop of just the face at 178x178
## But if you using the modified version it just resizes whatever you give it to 128x128, and then down to 16x16, and then upscales
#results show up in /content/Progressive-Face-Super-Resolution/result
!cp /content/Progressive-Face-Super-Resolution/result/*.png /usr/local/share/jupyter/nbextensions/google.colab/

/content/Progressive-Face-Super-Resolution
pre-trained model is loaded step:3, iteration:85000
 [0/1] Test progress... PSNR: 20.2209Test done, Average PSNR:20.2209, Average SSIM:0.5643, Average MS-SSIM:0.8670 


In [ ]:
%%html
<p>Input Images, second one is resized to 16x16 before input.</p>
<img src='https://i.imgur.com/2w2Y6li.png' style="width: 128px;" />
<img src='https://i.imgur.com/HpkjoOM.png' style="width: 128px;" />
<br>
<p>Predicted:</p>
<img src='/nbextensions/google.colab/0_predictedresults.png' />
<br>
<p>Actual:</p>
<img src='/nbextensions/google.colab/0_ground_truth.png' />